In [7]:
import pandas as pd
from datetime import datetime, timedelta

In [100]:
transaction_df = pd.read_csv('app_transaction.csv', parse_dates=['timestamp'])
analytics_df = pd.read_csv('app_analytics.csv', parse_dates=['timestamp'])

In [101]:
now = datetime.now()
time_24h = now - timedelta(hours=24)
time_3d = now - timedelta(days=3)
time_7d = now - timedelta(days=7)

In [102]:
def aggregate_signals(df, value_col):
    # Filter separately by time window
    df_24h = df[df['timestamp'] >= time_24h]
    df_3d = df[df['timestamp'] >= time_3d]
    df_7d = df[df['timestamp'] >= time_7d]

    # Aggregate each
    total = df.groupby("product_id")[value_col].sum().rename("total")
    agg_24h = df_24h.groupby("product_id")[value_col].sum().rename("24h")
    agg_3d = df_3d.groupby("product_id")[value_col].sum().rename("3d")
    agg_7d = df_7d.groupby("product_id")[value_col].sum().rename("7d")

    # Merge all
    agg = pd.concat([total, agg_7d, agg_3d, agg_24h], axis=1).fillna(0).reset_index()
    return agg


In [103]:
sales_agg = aggregate_signals(transaction_df, "purchase_count")
sales_agg.columns = ['product_id', 'total_sale', '7d_sale', '3d_sale', '24h_sale']

In [104]:
views_agg = aggregate_signals(analytics_df, "view_count")
views_agg.columns = ['product_id', 'total_view', '7d_view', '3d_view', '24h_view']


In [105]:
cart_agg = aggregate_signals(analytics_df, "addtocart_count")
cart_agg.columns = ['product_id', 'total_cart', '7d_cart', '3d_cart', '24h_cart']

In [106]:
wish_agg = aggregate_signals(analytics_df, "wishlist_count")
wish_agg.columns = ['product_id', 'total_wishlist', '7d_wish', '3d_wish', '24h_wish']


In [107]:
merged = sales_agg.merge(views_agg, on="product_id", how="left") \
                  .merge(cart_agg, on="product_id", how="left") \
                  .merge(wish_agg, on="product_id", how="left")

In [108]:
merged["weighted_sales"] = 0.6 * merged["24h_sale"] + 0.3 * merged["3d_sale"] + 0.1 * merged["7d_sale"]
merged["weighted_views"] = 0.6 * merged["24h_view"] + 0.4 * merged["3d_view"]
merged["weighted_cart"] = 0.6 * merged["24h_cart"] + 0.4 * merged["3d_cart"]
merged["weighted_wish"] = 0.6 * merged["3d_wish"] + 0.4 * merged["7d_wish"]


In [109]:
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

In [110]:
merged["norm_sales"] = normalize(merged["weighted_sales"])
merged["norm_views"] = normalize(merged["weighted_views"])
merged["norm_cart"] = normalize(merged["weighted_cart"])
merged["norm_wish"] = normalize(merged["weighted_wish"])


In [111]:
merged["trending_score"] = (
    0.40 * merged["norm_sales"] +
    0.30 * merged["norm_views"] +
    0.15 * merged["norm_cart"] +
    0.15 * merged["norm_wish"]
)

In [112]:
merged

,product_id,total_sale,7d_sale,3d_sale,24h_sale,total_view,7d_view,3d_view,24h_view,total_cart,...,24h_wish,weighted_sales,weighted_views,weighted_cart,weighted_wish,norm_sales,norm_views,norm_cart,norm_wish,trending_score
0,P001,107,107,51,17.0,879,879,227,181.0,420,...,50.0,36.2,199.4,94.8,144.6,0.596107,0.301087,0.340326,0.460109,0.448834
1,P002,59,53,24,0.0,1384,1384,811,413.0,685,...,92.0,12.5,572.2,264.6,243.4,0.019465,1.000000,1.000000,1.000000,0.607786
2,P003,112,112,44,10.0,1004,1004,324,213.0,410,...,27.0,30.4,257.4,54.4,134.0,0.454988,0.409824,0.183372,0.402186,0.392776
3,P004,69,69,16,0.0,823,823,275,79.0,328,...,29.0,11.7,157.4,41.6,111.6,0.000000,0.222347,0.133644,0.279781,0.128718
4,P005,75,75,26,5.0,555,527,99,0.0,316,...,0.0,18.3,39.6,14.8,64.4,0.160584,0.001500,0.029526,0.021858,0.072391
5,P006,75,75,37,10.0,1046,1046,427,233.0,414,...,38.0,24.6,310.6,116.6,141.2,0.313869,0.509561,0.425019,0.441530,0.408398
6,P007,99,99,34,0.0,1136,1136,624,168.0,600,...,33.0,20.1,350.4,169.0,229.6,0.204380,0.584177,0.628594,0.924590,0.489982
7,P008,139,139,56,10.0,644,644,256,61.0,281,...,54.0,36.7,139.0,72.6,130.6,0.608273,0.187852,0.254079,0.383607,0.395317
8,P009,79,79,16,2.0,583,583,234,0.0,214,...,0.0,13.9,93.6,31.2,128.4,0.053528,0.102737,0.093240,0.371585,0.121956
9,P010,117,117,49,38.0,584,584,197,53.0,230,...,16.0,49.2,110.6,66.8,117.0,0.912409,0.134608,0.231546,0.309290,0.486471


In [113]:
merged["is_trending"] = merged["trending_score"] >= 0.70


In [114]:
merged

,product_id,total_sale,7d_sale,3d_sale,24h_sale,total_view,7d_view,3d_view,24h_view,total_cart,...,weighted_sales,weighted_views,weighted_cart,weighted_wish,norm_sales,norm_views,norm_cart,norm_wish,trending_score,is_trending
0,P001,107,107,51,17.0,879,879,227,181.0,420,...,36.2,199.4,94.8,144.6,0.596107,0.301087,0.340326,0.460109,0.448834,False
1,P002,59,53,24,0.0,1384,1384,811,413.0,685,...,12.5,572.2,264.6,243.4,0.019465,1.000000,1.000000,1.000000,0.607786,False
2,P003,112,112,44,10.0,1004,1004,324,213.0,410,...,30.4,257.4,54.4,134.0,0.454988,0.409824,0.183372,0.402186,0.392776,False
3,P004,69,69,16,0.0,823,823,275,79.0,328,...,11.7,157.4,41.6,111.6,0.000000,0.222347,0.133644,0.279781,0.128718,False
4,P005,75,75,26,5.0,555,527,99,0.0,316,...,18.3,39.6,14.8,64.4,0.160584,0.001500,0.029526,0.021858,0.072391,False
5,P006,75,75,37,10.0,1046,1046,427,233.0,414,...,24.6,310.6,116.6,141.2,0.313869,0.509561,0.425019,0.441530,0.408398,False
6,P007,99,99,34,0.0,1136,1136,624,168.0,600,...,20.1,350.4,169.0,229.6,0.204380,0.584177,0.628594,0.924590,0.489982,False
7,P008,139,139,56,10.0,644,644,256,61.0,281,...,36.7,139.0,72.6,130.6,0.608273,0.187852,0.254079,0.383607,0.395317,False
8,P009,79,79,16,2.0,583,583,234,0.0,214,...,13.9,93.6,31.2,128.4,0.053528,0.102737,0.093240,0.371585,0.121956,False
9,P010,117,117,49,38.0,584,584,197,53.0,230,...,49.2,110.6,66.8,117.0,0.912409,0.134608,0.231546,0.309290,0.486471,False


In [115]:
top_trending = merged[merged["is_trending"]==True].sort_values(by="trending_score", ascending=False)

In [116]:
top_trending.to_csv("trending_products.csv", index=False)